In [48]:
import pathlib

import numpy as np
import pandas as pd
import tensorflow as tf

# loading and preprocessing the data

In [121]:
dataDirPath = pathlib.Path('kinect_good_preprocessed')

ds = tf.data.Dataset.from_generator(
    generator= lambda: (
        pd.read_csv(f).values[:,1:]
        for f in dataDirPath.iterdir()
    ),
    output_signature= tf.TensorSpec(shape= [None, 39], dtype= float)
)
ds = ds.batch(batch_size= 1)
ds = ds.map(lambda a    : (a[:,:,0::3],a[:,:,1::3],a[:,:,2::3]))
ds = ds.map(lambda x,y,z: (tf.concat([x,y], axis= -1), z ))
ds = ds.shuffle(buffer_size= 200)

trainDs = ds.take(150)
testDs  = ds.skip(150)

# defining our model

In [149]:
model = tf.keras.Sequential([
    
    tf.keras.Input(shape= [None,None,26]),

    tf.keras.layers.Conv1D(filters= 20, kernel_size= 5, padding= 'same'),
    tf.keras.layers.Conv1D(filters= 15, kernel_size= 5, padding= 'same'),
    tf.keras.layers.Conv1D(filters= 13, kernel_size= 5, padding= 'same'),
    # # tf.keras.layers.MaxPooling1D(pool_size= 5),

    tf.keras.layers.Dense(units= 13, activation= tf.keras.activations.sigmoid)
])
model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss     = tf.keras.losses.MeanSquaredError(),
)

# training our model

In [151]:
model.fit(trainDs, epochs= 20)

Epoch 1/20
150/150 [==============================] - 2s 2ms/step - loss: 0.0508
Epoch 2/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0252
Epoch 3/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0246
Epoch 4/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0251
Epoch 5/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0248
Epoch 6/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0252
Epoch 7/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0242
Epoch 8/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0247
Epoch 9/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0249
Epoch 10/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0239
Epoch 11/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0246
Epoch 12/20
150/150 [==============================] - 1s 2ms/step - loss: 0.0243
Epoch 13/20
150/150 [====

# testing our model

In [152]:
model.evaluate(testDs)

29/29 [==============================] - 1s 2ms/step - loss: 0.0239


0.023900873959064484